In [1]:
# stream data

import pandas as pd
import numpy as np

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

arrestData = pd.read_csv("PPAD.csv")

nPopulations = pd.read_csv("totalPopulation.csv", index_col = 'Id')

eduA = pd.read_csv("eduAttainment.csv")


#eduA.head(10)

In [2]:
# create dict of neighborhoods w default vals
nc = {}

for i in range(1, len(nPopulations)):
    nValue = nPopulations.loc[i, "Neighborhood"]
    if(nValue not in nc):
        nc[nValue] = 0
#print(nc)  

In [3]:
# create dict of neighborhoods : populations

numOfNeighborhoods = 0

for i in range(len(nPopulations)):
    numOfNeighborhoods += 1

pop = {}

for i in range(1, len(nPopulations)):
    val = nPopulations.loc[i, "Neighborhood"]
    if(val in nPopulations.values):
        pop[val] = nPopulations.get_value(nPopulations.loc[nPopulations['Neighborhood'] == val].index[0], 'Estimate; Total')

#pop

/ihome/crc/install/python/miniconda3-3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


In [4]:
# create dict of neighborhoods : crimes

for i in range(len(arrestData)):
    nValue = arrestData.loc[i, "INCIDENTNEIGHBORHOOD"]
    if(nValue in nc):
        nc[nValue] += 1
#nc   

In [5]:
# create dict of neighborhoods : arrest rates (per 100k)

for key, value in nc.items():
    nc[key] = (value / pop[key]) * 100000
#nc

In [6]:
# eliminate neighborhoods above average num of arrests

ncUpdated = {}

NATL_ARREST_RATE = 3152.6

for key, value in nc.items():
    if(value <= NATL_ARREST_RATE):
        ncUpdated[key] = value
#ncUpdated

In [9]:
# create dict of all neighborhoods : education rates
# sufficient education defined as completion of HS or greater

allGradRates = {}

for i in range(len(eduA)):
    totalVal = eduA.loc[i, "Estimate; Total:"]
    hsGradNum = eduA.loc[i, "Estimate; Total: - Regular high school diploma"]
    collegeUnderOne = eduA.loc[i, "Estimate; Total: - Some college, less than 1 year"]
    collegeOnePlus = eduA.loc[i, "Estimate; Total: - Some college, 1 or more years, no degree"]
    associates = eduA.loc[i, "Estimate; Total: - Associate's degree"]
    bachelors = eduA.loc[i, "Estimate; Total: - Bachelor's degree"]
    masters = eduA.loc[i, "Estimate; Total: - Master's degree"]
    professional = eduA.loc[i, "Estimate; Total: - Professional school degree"]
    doctorate = eduA.loc[i, "Estimate; Total: - Doctorate degree"]
    totalEdu = hsGradNum + collegeUnderOne + collegeOnePlus + associates + bachelors + masters + professional + doctorate
    neigh = eduA.loc[i, "Neighborhood"]
    if(neigh not in allGradRates):
        allGradRates[neigh] = totalEdu / totalVal

#allGradRates

In [11]:
# create dict of select neighborhoods :  education rate

selectGradRates = {}

for key in ncUpdated.keys():
    if(key not in selectGradRates):
        selectGradRates[key] = allGradRates[key]

#selectGradRates